In [7]:
from arithmetic_gf import gf_pow
from polynomial_gf import gf_poly_mul

def rs_generator_poly(nsym):
    '''Generate an irreducible generator polynomial (necessary to encode a message into Reed-Solomon)'''
    g = [1]
    for i in range(0, nsym):
        g = gf_poly_mul(g, [1, gf_pow(2, i)])
    return g

In [8]:
rs_generator_poly(4)

[1, 15, 54, 120, 64]

In [53]:
from arithmetic_gf import gf_mul

def gf_poly_div(dividend, divisor):
    '''Fast polynomial division by using Extended Synthetic Division and optimized for GF(2^p) computations
    (doesn't work with standard polynomials outside of this galois field, see the Wikipedia article for generic algorithm).'''
    # CAUTION: this function expects polynomials to follow the opposite convention at decoding:
    # the terms must go from the biggest to lowest degree (while most other functions here expect
    # a list from lowest to biggest degree). eg: 1 + 2x + 5x^2 = [5, 2, 1], NOT [1, 2, 5]

    msg_out = list(dividend) # Copy the dividend
    #normalizer = divisor[0] # precomputing for performance
    for i in range(0, len(dividend) - (len(divisor)-1)):
        #msg_out[i] /= normalizer # for general polynomial division (when polynomials are non-monic), the usual way of using
                                  # synthetic division is to divide the divisor g(x) with its leading coefficient, but not needed here.
        coef = msg_out[i] # precaching
        if coef != 0: # log(0) is undefined, so we need to avoid that case explicitly (and it's also a good optimization).
            for j in range(1, len(divisor)): # in synthetic division, we always skip the first coefficient of the divisior,
                                              # because it's only used to normalize the dividend coefficient
                if divisor[j] != 0: # log(0) is undefined
                    msg_out[i + j] ^= gf_mul(divisor[j], coef) # equivalent to the more mathematically correct
                                                               # (but xoring directly is faster): msg_out[i + j] += -divisor[j] * coef

    # The resulting msg_out contains both the quotient and the remainder, the remainder being the size of the divisor
    # (the remainder has necessarily the same degree as the divisor -- not length but degree == length-1 -- since it's
    # what we couldn't divide from the dividend), so we compute the index where this separation is, and return the quotient and remainder.
    separator = -(len(divisor)-1)
    return msg_out[:separator], msg_out[separator:] # return quotient, remainder.

In [54]:
def poly_to_str(poly):
    # Theinput is a polynomial in descending order
    return "+".join([f"{x:x}x^{len(poly)-ind-1}" for ind, x in enumerate(poly) if x != 0])

In [58]:
"""
Present the first n generator polynomials, all coefficients are in base 16
"""
n = 15
for i in range(n):
    print(f"{poly_to_str(rs_generator_poly(i))}")

1x^0
1x^1+1x^0
1x^2+3x^1+2x^0
1x^3+7x^2+ex^1+8x^0
1x^4+fx^3+36x^2+78x^1+40x^0
1x^5+1fx^4+c6x^3+3fx^2+93x^1+74x^0
1x^6+3fx^5+1x^4+dax^3+20x^2+e3x^1+26x^0
1x^7+7fx^6+7ax^5+9ax^4+a4x^3+bx^2+44x^1+75x^0
1x^8+ffx^7+bx^6+51x^5+36x^4+efx^3+adx^2+c8x^1+18x^0
1x^9+e2x^8+cfx^7+9ex^6+f5x^5+ebx^4+a4x^3+e8x^2+c5x^1+25x^0
1x^10+d8x^9+c2x^8+9fx^7+6fx^6+c7x^5+5ex^4+5fx^3+71x^2+9dx^1+c1x^0
1x^11+acx^10+82x^9+a3x^8+32x^7+7bx^6+dbx^5+a2x^4+f8x^3+90x^2+74x^1+a0x^0
1x^12+44x^11+77x^10+43x^9+76x^8+dcx^7+1fx^6+7x^5+54x^4+5cx^3+7fx^2+d5x^1+61x^0
1x^13+89x^12+49x^11+e3x^10+11x^9+b1x^8+11x^7+34x^6+dx^5+2ex^4+2bx^3+53x^2+84x^1+78x^0
1x^14+ex^13+36x^12+72x^11+46x^10+aex^9+97x^8+2bx^7+9ex^6+c3x^5+7fx^4+a6x^3+d2x^2+eax^1+a3x^0


In [55]:
"""
divide h12x^6 + h34x^5 + h56x^4 (Every coefficient is in base 16)
"""
dividend = [0x12, 0x34, 0x56, 0, 0, 0, 0] # message polynomial with every degree d => d+nsym
divisor = rs_generator_poly(4)
pdiv = gf_poly_div(dividend, divisor)

sdividend = poly_to_str(dividend)
sdivisor = poly_to_str(divisor)
print(f"Dividend: {sdividend}, \tDivisor: {sdivisor}")

quotient = poly_to_str(pdiv[0])
remainder = poly_to_str(pdiv[1])
print(f"Quotient: {quotient},\tRemainder: {remainder}")

Dividend: 12x^6+34x^5+56x^4, 	Divisor: 1x^4+fx^3+36x^2+78x^1+40x^0
Quotient: 12x^2+dax^1+dfx^0,	Remainder: 37x^3+e6x^2+78x^1+d9x^0
